**Outline**
* [Get the clustering result](#getcluster)

* [Replace with the new cluster](#replace)
* [Team level](#team)
    * [Performance + weapon + class](#team_all)
    * [Weapon + class](#team_Wc)    
    * [Team Distance](#team_dist)      
    * [Find NN for teams](#findnn) 
    * [Find the team with the highest win-loss](#winloss) 
    * [Find the team with the highest learning rate](#learning)
* [Players to Players](#players)
    * [Players weapon + class](#player_wc) 
    * [Players distance](#player_dist) 
    * [Players to players mapping](#player_map) 
* [Final Recommendation](#rec)


In [1]:
import numpy as np
#from py_pcha.PCHA import PCHA
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn import mixture
from sklearn.cluster import KMeans
import os
from scipy.spatial.distance import squareform, pdist,cdist
import textwrap
import json
import itertools

import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [619]:
### Read data
os.chdir('/Users/yuecui/Desktop/Destiny/final_team')
final_cluster_clean_id=pd.read_csv("final_cluster_clean_5_10.csv")
final_cluster_clean=final_cluster_clean_id.drop(["playersID","teamID"],axis=1)
newtable = pd.read_csv("final_label_5_15.csv")
newset = newtable[["playersID","teamID","performance_cluster","weapon_pro_cluster","Character_Class"]]

,playersID,teamID,performance_cluster,weapon_pro_cluster,Character_Class
0,2305843009260294880,2305843009260294880230584300926065511223058430...,4,6,Warlock
1,2305843009260301047,2305843009260301047230584300926407495723058430...,3,6,Warlock
2,2305843009260304802,2305843009260304802230584300926112491023058430...,4,1,Hunter
3,2305843009260306096,2305843009260306096230584300926893816923058430...,4,3,Warlock
4,2305843009260307017,2305843009260307017230584300926409646823058430...,0,1,Hunter


In [48]:
winloss = pd.read_csv("WinRatio_upto15.csv").drop("Unnamed: 0",axis=1)
winloss.head()

,teamID,Wins
0,2305843009260293961230584300926060138823058430...,0.0
1,2305843009260293961230584300926228396323058430...,1.0
2,2305843009260293963230584300926039475323058430...,1.0
3,2305843009260293963230584300926043395823058430...,1.0
4,2305843009260293963230584300926043395823058430...,1.0


In [71]:
learning = pd.read_csv("team_learning_rates.csv")
learning.head()

,teamID,lr_Grenade,lr_Melee,lr_Super,lr_assists,lr_avg_score_per_life,lr_KDA,lr_efficiency
0,2305843009260293961230584300926060138823058430...,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2305843009260293961230584300926228396323058430...,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2305843009260293963230584300926039475323058430...,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2305843009260293963230584300926043395823058430...,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2305843009260293963230584300926043395823058430...,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [328]:
playstyle = pd.read_csv("per_recomendation_data_6_8.csv")

In [385]:
import json
with open('weapon_info.json', 'r') as fp:
    weapon_list = json.load(fp)
with open('player_weapon_sort_frequency.json', 'r') as fp:
    referenceid_list = json.load(fp)

def weapon_from_team(team):
    team_id=str(team)
    team_id_list=[team_id[0:19],team_id[19:38],team_id[38:57],team_id[57:76]]
    final_dic={}
    for teamid in team_id_list:
        weapon_l=[]
        for re in referenceid_list[teamid]:
            weapon_l.append(weapon_list[str(re)])
        final_dic[teamid]=weapon_l[:3] #top three weapons    
    return final_dic

In [3]:
def my_scaler(x):
    """standardize the predictors"""
    
    new_x = pd.DataFrame(scale(x, axis=0, with_mean=True, with_std=True, copy=True))
    new_x.columns = x.columns
    
    return new_x

def my_normal(x):
    """normalize the predictors"""
    
    scaler = MinMaxScaler() 
    scaled_values = scaler.fit_transform(x)
    new_x = pd.DataFrame(scaled_values)
    new_x.columns = x.columns

    return new_x

In [4]:
def cluster(dt,cluster_content,scale, cluster_number, cluster_method):
    player_feature_df=dt
    performance_cluster=[]
    weapon_cluster=[]
    
    if scale=="MinMax":
        min_max_scaler = preprocessing.MinMaxScaler()
        player_feature_df = my_normal(player_feature_df)
        
    if scale=="Standardization":
        player_feature_df = my_scaler(player_feature_df)
            
    if cluster_content=="performance":
        cluster_dt=player_feature_df[['Ability', 'Grenade', 'Melee','Super', 'assists']]
        
    if cluster_content=="weapon":
        cluster_dt=player_feature_df[['Pro Auto Rifle',
                                      'Pro Fusion Rifle', 'Pro Grenade Launcher', 'Pro Hand Cannon',
                                      'Pro Linear Fusion Rifle', 'Pro Pulse Rifle', 'Pro Rocket Launcher',
                                      'Pro Scout Rifle', 'Pro Shotgun', 'Pro Sidearm', 'Pro Sniper Rifle',
                                      'Pro Submachine Gun', 'Pro Sword', 'Pro Trace Rifle']]   
        
    if cluster_method=="GMM":
        clf = mixture.GaussianMixture(n_components=cluster_number, covariance_type='full')
        clf.fit(cluster_dt)
        cluster_dt = cluster_dt.copy()
        if cluster_content=="performance":
            performance_cluster = clf.predict(cluster_dt)
        if cluster_content=="weapon":
            weapon_cluster = clf.predict(cluster_dt) 
            
    if cluster_method=="KMeans":
        model = KMeans(n_clusters=cluster_number, n_init=30)
        model.fit(cluster_dt)
        cluster_dt = cluster_dt.copy()
        if cluster_content=="performance":
            performance_cluster = model.predict(cluster_dt)
        if cluster_content=="weapon":
            weapon_cluster = model.predict(cluster_dt)        
            
    if cluster_method=="AA":
        cluster_matrix = cluster_dt.as_matrix()
        XC, S, C, SSE, varexpl = PCHA(cluster_matrix, noc=cluster_number, delta=0.1)
        XC_df=pd.DataFrame(XC)
        if cluster_content=="performance":
            performance_cluster = XC_df.idxmax(axis=1)
        if cluster_content=="weapon":
            weapon_cluster = XC_df.idxmax(axis=1)       
            
    return performance_cluster,weapon_cluster

### <a id='getcluster'>Get the clustering result</a>

In [5]:
### GMM Use standarization
performance_cluster_std = cluster(dt=final_cluster_clean,cluster_content="performance",scale="Standardization", 
        cluster_number = 5, cluster_method="GMM")
performance_cluster_std = performance_cluster_std[0]

In [229]:
weapon_cluster_std = cluster(dt=final_cluster_clean,cluster_content="weapon",scale="Standardization", 
        cluster_number = 7, cluster_method="GMM")
weapon_cluster_std = weapon_cluster_std[1]

### <a id='replace'>Replace with the new clusters</a>

In [620]:
newset["performance_cluster"] = performance_cluster_std
newset["weapon_pro_cluster"] = weapon_cluster_std
newset.head()

/Users/yuecui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/yuecui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,playersID,teamID,performance_cluster,weapon_pro_cluster,Character_Class
0,2305843009260294880,2305843009260294880230584300926065511223058430...,0,4,Warlock
1,2305843009260301047,2305843009260301047230584300926407495723058430...,0,4,Warlock
2,2305843009260304802,2305843009260304802230584300926112491023058430...,0,4,Hunter
3,2305843009260306096,2305843009260306096230584300926893816923058430...,0,5,Warlock
4,2305843009260307017,2305843009260307017230584300926409646823058430...,0,4,Hunter


### <a id='team'>Team level</a>

#### <a id='team_all'>Performance + Weapon + Class  --> Weapon</a>

In [581]:
def teamall(df):
    df["values"] = 1
    table_weapon = df.pivot_table(index="teamID", columns="weapon_pro_cluster", values = "values",aggfunc=sum)
    table_char = df.pivot_table(index="teamID", columns="Character_Class", values = "values",aggfunc=sum)
    table_per = df.pivot_table(index="teamID", columns="performance_cluster", values = "values",aggfunc=sum)
    table_weapon = table_weapon.fillna(0)
    table_char = table_char.fillna(0)
    table_per = table_per.fillna(0)
    finalresult = pd.concat([table_weapon, table_char,table_per], axis=1).reset_index()
    return finalresult


In [607]:
def changeweight(df,char_weight):
    df[["Hunter","Titan","Warlock"]] = df[["Hunter","Titan","Warlock"]] * char_weight
    return df

all_dim = teamall(newset)
all_dim = changeweight(all_dim,1.5)
all_dim.columns = ['teamID', 'Weapon_0', 'Weapon_1',"Weapon_2","Weapon_3","Weapon_4","Weapon_5","Weapon_6",
                  "Hunter","Titan","Warlock",
                  "Per_0","Per_1","Per_2","Per_3","Per_4"]

/Users/yuecui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



#### <a id='team_wc'>Weapon + Classs --> Playstyle</a>

In [583]:
def teamweaponandchar(df):
    df["values"] = 1
    table_weapon = df.pivot_table(index="teamID", columns="weapon_pro_cluster", values = "values",aggfunc=sum)
    table_char = df.pivot_table(index="teamID", columns="Character_Class", values = "values",aggfunc=sum)
    table_weapon = table_weapon.fillna(0)
    table_char = table_char.fillna(0)
    finalresult = pd.concat([table_weapon, table_char], axis=1).reset_index()
    return finalresult

weapon_and_char = teamweaponandchar(newset)
weapon_and_char = changeweight(weapon_and_char,1.5)
weapon_and_char.head()

/Users/yuecui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,teamID,0,1,2,3,4,5,6,Hunter,Titan,Warlock
0,2305843009260294880230584300926065511223058430...,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,3.0
1,2305843009260301047230584300926407495723058430...,0.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,1.5,4.5
2,2305843009260304802230584300926112491023058430...,0.0,2.0,0.0,0.0,2.0,0.0,0.0,4.5,1.5,0.0
3,2305843009260306096230584300926893816923058430...,0.0,0.0,0.0,0.0,2.0,2.0,0.0,1.5,3.0,1.5
4,2305843009260307017230584300926409646823058430...,0.0,0.0,1.0,0.0,3.0,0.0,0.0,3.0,1.5,1.5


#### <a id='team_dist'>Team Distance </a>

In [584]:
def get_dist(dimensions):
    df = pd.DataFrame(squareform(pdist(dimensions.iloc[:, 1:],metric = 'cityblock')), 
                     columns=dimensions.teamID.unique(), index=dimensions.teamID.unique())\
                     .reset_index()
    index = df.iloc[:,0]
    df = df.iloc[:,1:]
    df.values[[np.arange(len(df))]*2] = 99999
    df.insert(0,"index",index)
    return df

In [608]:
dist_all = get_dist(all_dim)

#### <a id='findnn'>Find NN for teams </a>

In [609]:
def findnn(distance_df,n):
    idx = np.argsort(distance_df, axis=0)[:n]
    idx = idx.drop('index', 1)
    dfList = distance_df['index'].tolist()
    dfList = distance_df
    IdxDict=dict(enumerate(dfList))
    idx.replace(IdxDict,inplace=True)
    return idx

In [610]:
### Using weapon+class to recommend performance
resultteams = findnn(dist_all,5) 
resultteams

,2305843009260294880230584300926065511223058430092657851422305843009265925242,2305843009260301047230584300926407495723058430092908951392305843009290965137,2305843009260304802230584300926112491023058430092627147842305843009269893032,2305843009260306096230584300926893816923058430092689547272305843009284689157,2305843009260307017230584300926409646823058430092643115142305843009267824705,2305843009260309699230584300926052035523058430092659082862305843009283578425,2305843009260313050230584300926156342923058430092662893912305843009294828645,2305843009260314368230584300928558757523058430092859879742305843009294145614,2305843009260314920230584300926119480823058430092642156462305843009268588711,2305843009260314920230584300926119480923058430092642156462305843009290826059,...,2305843009279091232230584300928338672623058430092909563352305843009291488894,2305843009279584838230584300928385805923058430092951660482305843009301313577,2305843009283349910230584300928392994323058430092901878302305843009290245928,2305843009283494720230584300928388544523058430093100742702305843009320594027,2305843009285127105230584300928628906823058430093009017302305843009320370813,2305843009285378434230584300928596688723058430093062291572305843009307125061,2305843009285938200230584300930979771623058430093105039912305843009329993983,2305843009289908109230584300929226667423058430092942876532305843009301365345,2305843009294799163230584300930078323223058430093098446092305843009318975162,2305843009309609567230584300931633481523058430093171760282305843009331015583
0,2305843009266257500230584300926743186723058430...,2305843009260767734230584300926137522323058430...,2305843009260699219230584300926074932523058430...,2305843009261921349230584300926260813123058430...,2305843009262029854230584300926442821823058430...,2305843009262283456230584300926797848923058430...,2305843009261291110230584300926695271123058430...,2305843009265343488230584300926638368923058430...,2305843009260314920230584300926119480823058430...,2305843009260341676230584300926644509623058430...,...,2305843009262841364230584300927055588923058430...,2305843009262665654230584300926358275223058430...,2305843009261106168230584300926215561823058430...,2305843009262903674230584300926354708623058430...,2305843009261973826230584300926500092823058430...,2305843009261975516230584300926861355823058430...,2305843009260900406230584300926143781823058430...,2305843009261902514230584300926365080223058430...,2305843009264635080230584300926616542923058430...,2305843009279024806230584300929015840323058430...
1,2305843009260505665230584300926333160323058430...,2305843009264987063230584300926933800223058430...,2305843009260589621230584300926136938123058430...,2305843009267450310230584300926915976923058430...,2305843009263378400230584300926595071423058430...,2305843009260688946230584300926280128123058430...,2305843009266657265230584300926710223823058430...,2305843009261612596230584300926211999423058430...,2305843009262119965230584300926370941723058430...,2305843009260314368230584300928558757523058430...,...,2305843009260475640230584300926534777523058430...,2305843009262811456230584300926500756823058430...,2305843009260596484230584300926133553923058430...,2305843009269544537230584300927312415823058430...,2305843009272948179230584300927824492623058430...,2305843009264549180230584300926912444823058430...,2305843009260689568230584300926071110223058430...,2305843009260718299230584300926342443823058430...,2305843009262480257230584300926264821423058430...,2305843009262242765230584300926871294123058430...
2,2305843009265887920230584300926595080023058430...,2305843009261675675230584300926266471223058430...,2305843009264525649230584300927932113023058430...,2305843009261231826230584300926223946923058430...,2305843009262548545230584300926360338223058430...,2305843009283349910230584300928392994323058430...,2305843009263622849230584300926442705323058430...,2305843009262226823230584300926546495923058430...,23058430092603416762305843009266445096230

#### <a id='target'>Choose the target team </a>

In [611]:
def choosetarget(data):
    target = textwrap.wrap(data,19)
    testteam = resultteams.iloc[:,resultteams.\
                                columns.\
                                get_loc(data)]
    return testteam

In [612]:
#targetteam = "2305843009264404279230584300927803779723058430092902852642305843009291975432"
targetteam = "2305843009262414917230584300926683841023058430092716892462305843009290855561"
testteam = choosetarget(targetteam)
testteam

0    2305843009260843487230584300926168174023058430...
1    2305843009260997357230584300926177467723058430...
2    2305843009260821448230584300926304426023058430...
3    2305843009261992066230584300926744760223058430...
4    2305843009262029854230584300927059679223058430...
Name: 2305843009262414917230584300926683841023058430092716892462305843009290855561, dtype: object

#### <a id='winloss'>Find the team with highest win-loss ratio </a>

In [591]:
def getWinloss(testteam):
    df = pd.DataFrame(np.array(testeam))
    df.columns = ['teamID']
    df_merge = pd.merge(df,winloss,"left",on="teamID")
    finalteam = df_merge.sort_values("Wins", ascending = False).iloc[0,0]
    result = textwrap.wrap(finalteam,19)
    return result,finalteam

In [592]:
test_win =getWinloss(testteam)

In [593]:
test_win

(['2305843009260618250',
  '2305843009260898955',
  '2305843009262768291',
  '2305843009265840141'],
 '2305843009260618250230584300926089895523058430092627682912305843009265840141')

#### <a id='learning'>Find the team with highest learning rate </a>

In [613]:
def getLearning(data, metrics):
    df = pd.DataFrame(np.array(data))
    df.columns = ['teamID']
    df_merge = pd.merge(df,learning,"left",on="teamID")
    finalteam = df_merge.sort_values(metrics, ascending = False).iloc[0,0]
    #finaltemp = df_merge.sort_values(metrics, ascending = False)
    result = textwrap.wrap(finalteam,19)
    return result,finalteam

In [673]:
test_lr = getLearning(testteam, "lr_KDA")

In [674]:
test_lr

(['2305843009260821448',
  '2305843009263044260',
  '2305843009264842296',
  '2305843009268897947'],
 '2305843009260821448230584300926304426023058430092648422962305843009268897947')

### <a id='players'>Players to Players</a>

#### <a id='player_wc'>Players weapon + class </a>

In [594]:
def getplayerdist(df):
    df["values"] = 1
    table_weapon = df.pivot_table(index="playersID", columns="weapon_pro_cluster", values = "values",aggfunc=sum)
    table_char = df.pivot_table(index="playersID", columns="Character_Class", values = "values",aggfunc=sum)
    #table_per = df.pivot_table(index="playersID", columns="performance_cluster", values = "values",aggfunc=sum)
    table_weapon = table_weapon.fillna(0)
    table_char = table_char.fillna(0)
    #table_per = table_per.fillna(0)
    finalresult = pd.concat([table_weapon, table_char], axis=1).reset_index()
    return finalresult

In [595]:
playersdist = getplayerdist(newset)

/Users/yuecui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [596]:
### Increase weights of class
def changeweight(df,weight):
    df[["Hunter","Titan","Warlock"]] = df[["Hunter","Titan","Warlock"]]*weight
    return df
playersdist_new = changeweight(playersdist,2)

#### <a id='player_all'>Players playstyle + weapon + class </a>

In [629]:
def getplayerdist_all(df):
    df["values"] = 1
    table_weapon = df.pivot_table(index="playersID", columns="weapon_pro_cluster", values = "values",aggfunc=sum)
    table_char = df.pivot_table(index="playersID", columns="Character_Class", values = "values",aggfunc=sum)
    table_per = df.pivot_table(index="playersID", columns="performance_cluster", values = "values",aggfunc=sum)
    table_weapon = table_weapon.fillna(0)
    table_char = table_char.fillna(0)
    table_per = table_per.fillna(0)
    finalresult = pd.concat([table_weapon, table_char,table_per], axis=1).reset_index()
    return finalresult

In [630]:
playersdist = getplayerdist_all(newset)

/Users/yuecui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [631]:
### Increase weights of class
def changeweight(df,weight):
    df[["Hunter","Titan","Warlock"]] = df[["Hunter","Titan","Warlock"]]*weight
    return df
playersdist_new = changeweight(playersdist,2)
playersdist_new.head()

,playersID,0,1,2,3,4,5,6,Hunter,Titan,Warlock,0,1,2,3,4
0,2305843009260294880,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
1,2305843009260301047,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
2,2305843009260304802,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2305843009260306096,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
4,2305843009260307017,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [632]:
playersdist_new.columns = ['playersID', 'Weapon_0', 'Weapon_1',"Weapon_2","Weapon_3","Weapon_4","Weapon_5","Weapon_6",
                  "Hunter","Titan","Warlock",
                  "Per_0","Per_1","Per_2","Per_3","Per_4"]
playersdist_new.head()

,playersID,Weapon_0,Weapon_1,Weapon_2,Weapon_3,Weapon_4,Weapon_5,Weapon_6,Hunter,Titan,Warlock,Per_0,Per_1,Per_2,Per_3,Per_4
0,2305843009260294880,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
1,2305843009260301047,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
2,2305843009260304802,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2305843009260306096,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
4,2305843009260307017,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


#### <a id='player_dist'>Players distance </a>

In [641]:
def getPlayerDistances(team1, team2, playerclusters_df):
    playerclusters_df1 = playerclusters_df[playerclusters_df.playersID.isin(team1)]
    playerclusters_df2 = playerclusters_df[playerclusters_df.playersID.isin(team2)]
    
    playerdist = pd.DataFrame(cdist(playerclusters_df1.iloc[:, 1:], playerclusters_df2.iloc[:, 1:],
                                            metric = 'cityblock'), \
                     columns=playerclusters_df2.playersID, index=playerclusters_df1.playersID)\
                    .reset_index()
    playerdist.index.names = ['index']

    return playerdist

In [642]:
def getDistmat(LrorWin):
    tm2 = LrorWin[0]
    tm1 = target
    distmat = getPlayerDistances(tm1, tm2, playersdist_new)
    return distmat

In [675]:
distmat = getDistmat(test_lr)
distmat.head()

playersID,playersID,2305843009260821448,2305843009263044260,2305843009264842296,2305843009268897947
index,,,,,
0,2305843009264404279,6.0,4.0,8.0,2.0
1,2305843009278037797,8.0,6.0,8.0,4.0
2,2305843009290285264,8.0,6.0,8.0,4.0
3,2305843009291975432,6.0,4.0,4.0,6.0


#### <a id='player_map'>Players to players mapping </a>

In [676]:
def getDistList(mapping, dist_df):
    distlist = []
#     for k,v in mapping.items():
#         dist = dist_df[dist_df.playersID == k, v]
#         distlist.append(dist)
    distlist = list(dist_df[dist_df["playersID"] == tup[0]][tup[1]] for tup in mapping)
    newdistlist = [item for items in distlist for item in items]
    return newdistlist
    

In [677]:
def sortByDist(mapped):
    mapped.sort(key = lambda y: y[1])
    #print(mapped)
    return mapped

In [678]:
''' dist_df - is the created distance matrix with players of team 1 as rows, team 2 as columns
    numToSum - The number of best matched players to consider when deciding the best possible mapping.
                (eg. All 4 could result in 4 players not matched well, but if used to the first 2 could match perfectly
                but next 2 poorly, but at least highly relevant for fist 2)
'''
def getPlayerMapping(dist_df, numToSum):
    mapping = {}
    permuts = list(itertools.permutations(dist_df.playersID))
    #for tm1player in dist_df.playersID:
    team2 = list(dist_df.iloc[:, 1:].columns.values)
    newdf = pd.DataFrame({"Team1":permuts})
    newdf["Mapping"] = newdf.apply(lambda x: list(list(zip(x["Team1"], team2))[:]), axis = 1)
    newdf["Dists"] = newdf.apply(lambda x: getDistList(x["Mapping"], dist_df), axis = 1)
#   print(newdf)
    newdf["MapWithDist"] = newdf.apply(lambda x: list(list(zip(x["Mapping"], x["Dists"]))[:]), axis = 1)
    #print(newdf["MapWithDist"][0], "\n\n")
    newdf["SortedByDist"] = newdf["MapWithDist"].apply(lambda x: sortByDist(x))
    #print(newdf)
    newdf["PureMappingFinal"] = newdf["SortedByDist"].apply(lambda x: [(tup[0][0], tup[0][1]) for tup in x])
    #print(newdf["PureMappingFinal"][0])
    newdf["SumDist"] = newdf.apply(lambda x: sum(tup[1] for tup in x["SortedByDist"][0:(numToSum - 1)]), axis = 1)
    #print(newdf)
    newdf["AllPlayersDist"] = newdf.apply(lambda x: sum(tup[1] for tup in x["SortedByDist"]), axis = 1)
    #print(newdf)
    mindist_idx = newdf.SumDist.idxmin
    finalmapping = newdf["PureMappingFinal"][mindist_idx]
    finalsum = newdf["SumDist"][mindist_idx]
    finaltotalsum = newdf["AllPlayersDist"][mindist_idx]
    
    #ONLY NEED THE FIRST ELEMENT, BUT SECOND TELLS US WHAT THE TOTAL RELEVANT DISTANCE WAS, THIRD TELLS US TOTAL
    #DISTANCE FOR ALL 4 PLAYERS
    return finalmapping

        

In [679]:
playerresult = getPlayerMapping(distmat,2)

In [680]:
playerresult


[(2305843009264404279, 2305843009268897947),
 (2305843009291975432, 2305843009264842296),
 (2305843009290285264, 2305843009263044260),
 (2305843009278037797, 2305843009260821448)]

### <a id='rec'>Final Recommendation</a>

#### <a id='rec'>Recommendation for playstyle based on Weapon + Class based on win-loss</a>

In [764]:
playstyle.head()

,teamID,playersID,Grenade,Melee,assists,Ability,Super
0,2305843009260294880230584300926065511223058430...,2305843009260294880,0.888889,0.111111,4.444444,0.000000,1.777778
1,2305843009260294880230584300926065511223058430...,2305843009260655112,1.222222,0.000000,7.000000,0.000000,2.000000
2,2305843009260294880230584300926065511223058430...,2305843009265785142,0.444444,0.111111,5.444444,0.222222,0.666667
3,2305843009260294880230584300926065511223058430...,2305843009265925242,0.666667,0.666667,5.666667,0.000000,1.333333
4,2305843009260301047230584300926407495723058430...,2305843009260301047,0.300000,0.700000,3.100000,0.000000,0.700000


In [800]:
def getPlaystyle(data, test):
    df = pd.DataFrame(list(data))
    df.columns = ['targetplayersID','playersID']
    
    df['teamID'] = test[1]
    df['targetteamID'] = targetteam
    
    
    #df_merge = pd.merge(df,playstyle,"left",on=["teamID","playersID"])
    
    playstyle.columns = ["TEAMID","PLAYERSID","Grenade","Melee","assists","Ability","Super"]
    playstyle["TEAMID"] = str(playstyle["TEAMID"])
    
    df_merge2= pd.merge(df,playstyle,left_on = ["targetplayersID","targetteamID"], right_on = ["PLAYERSID","TEAMID"],
                       how = "left")
    
    return df_merge2

In [805]:
getPlaystyle(playerresult,test_lr)

,targetplayersID,playersID,teamID,targetteamID,TEAMID,PLAYERSID,Grenade,Melee,assists,Ability,Super
0,2305843009264404279,2305843009268897947,2305843009260821448230584300926304426023058430...,2305843009262414917230584300926683841023058430...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2305843009291975432,2305843009264842296,2305843009260821448230584300926304426023058430...,2305843009262414917230584300926683841023058430...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2305843009290285264,2305843009263044260,2305843009260821448230584300926304426023058430...,2305843009262414917230584300926683841023058430...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2305843009278037797,2305843009260821448,2305843009260821448230584300926304426023058430...,2305843009262414917230584300926683841023058430...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [803]:
getPlaystyle(playerresult,test_lr).iloc[0,2]

'2305843009260821448230584300926304426023058430092648422962305843009268897947'

In [804]:
getPlaystyle(playerresult,test_lr).iloc[0,3]

'2305843009262414917230584300926683841023058430092716892462305843009290855561'

In [807]:
type(df["playersID"][0])

numpy.int64

In [796]:
df.head()

,targetplayersID,playersID,teamID,targetteamID
0,2305843009264404279,2305843009268897947,2305843009260821448230584300926304426023058430...,2305843009262414917230584300926683841023058430...
1,2305843009291975432,2305843009264842296,2305843009260821448230584300926304426023058430...,2305843009262414917230584300926683841023058430...
2,2305843009290285264,2305843009263044260,2305843009260821448230584300926304426023058430...,2305843009262414917230584300926683841023058430...
3,2305843009278037797,2305843009260821448,2305843009260821448230584300926304426023058430...,2305843009262414917230584300926683841023058430...


In [792]:
playstyle.head()

,TEAMID,PLAYERSID,Grenade,Melee,assists,Ability,Super
0,0 0 230584300926029488023058430092...,2305843009260294880,0.888889,0.111111,4.444444,0.000000,1.777778
1,0 0 230584300926029488023058430092...,2305843009260655112,1.222222,0.000000,7.000000,0.000000,2.000000
2,0 0 230584300926029488023058430092...,2305843009265785142,0.444444,0.111111,5.444444,0.222222,0.666667
3,0 0 230584300926029488023058430092...,2305843009265925242,0.666667,0.666667,5.666667,0.000000,1.333333
4,0 0 230584300926029488023058430092...,2305843009260301047,0.300000,0.700000,3.100000,0.000000,0.700000


In [752]:
playstyle[playstyle.playersID == 2305843009268897947]

,teamID,playersID,Grenade,Melee,assists,Ability,Super
723,2305843009260821448230584300926304426023058430...,2305843009268897947,0.166667,0.25,3.333333,0.0,0.083333


#### <a id='rec'>Recommendation for weapon based on Playstyle + Weapon + Class based on learning rate</a>

In [722]:
#create a dataframe with columns: playerID, weapon
new_dt=getPlaystyle(playerresult,test_lr)
new_dt=new_dt[["teamID","playersID"]]
new_dt

,teamID,playersID
0,2305843009260821448230584300926304426023058430...,2305843009268897947
1,2305843009260821448230584300926304426023058430...,2305843009264842296
2,2305843009260821448230584300926304426023058430...,2305843009263044260
3,2305843009260821448230584300926304426023058430...,2305843009260821448


In [723]:
weapon_1=[]
weapon_2=[]
weapon_1=[]

In [724]:
for playersID in list(new_dt["playersID"]):
    weapon_1.append(result[str(playersID)][0])
    weapon_2.append(result[str(playersID)][1])
    weapon_3.append(result[str(playersID)][2])


In [725]:
new_dt["weapon_1"]=weapon_1
new_dt["weapon_2"]=weapon_2
new_dt["weapon_3"]=weapon_3

ValueError: Length of values does not match length of index

In [698]:
new_dt

,teamID,playersID,weapon_1,weapon_2,weapon_3
0,2305843009260821448230584300926304426023058430...,2305843009268897947,"[Galliard-42, Auto Rifle, Legendary]","[MIDA Multi-Tool, Scout Rifle, Exotic]","[Last Hope, Sidearm, Legendary]"
1,2305843009260821448230584300926304426023058430...,2305843009264842296,"[Lincoln Green, Pulse Rifle, Legendary]","[Eternity's Edge, Sword, Legendary]","[The Old Fashioned, Hand Cannon, Legendary]"
2,2305843009260821448230584300926304426023058430...,2305843009263044260,"[MIDA Multi-Tool, Scout Rifle, Exotic]","[Prosecutor, Auto Rifle, Legendary]","[Sins of the Past, Rocket Launcher, Legendary]"
3,2305843009260821448230584300926304426023058430...,2305843009260821448,"[Vigilance Wing, Pulse Rifle, Exotic]","[The Last Dance, Sidearm, Legendary]","[Sins of the Past, Rocket Launcher, Legendary]"


In [702]:
def getWeapondf(data, test):
    df = pd.DataFrame(list(data))
    df.columns = ['targetplayersID','playersID']
    df['teamID'] = test[1]
    return df

In [726]:
weapondf = getWeapondf(playerresult,test_lr)
weapondf

,targetplayersID,playersID,teamID
0,2305843009264404279,2305843009268897947,2305843009260821448230584300926304426023058430...
1,2305843009291975432,2305843009264842296,2305843009260821448230584300926304426023058430...
2,2305843009290285264,2305843009263044260,2305843009260821448230584300926304426023058430...
3,2305843009278037797,2305843009260821448,2305843009260821448230584300926304426023058430...


In [721]:
def getWeapon(df):
    weapon_1=[]
    weapon_2=[]
    weapon_3=[] 
    for playersID in list(df["playersID"]):
        weapon_1.append(result[str(playersID)][0])
        weapon_2.append(result[str(playersID)][1])
        weapon_3.append(result[str(playersID)][2])
    df["weapon_1"]=weapon_1
    df["weapon_2"]=weapon_2
    df["weapon_3"]=weapon_3
    return df
weaponrec = getWeapon(weapondf)
weaponrec

,targetplayersID,playersID,teamID,weapon_1,weapon_2,weapon_3
0,2305843009264404279,2305843009268897947,2305843009260821448230584300926304426023058430...,"[Galliard-42, Auto Rifle, Legendary]","[MIDA Multi-Tool, Scout Rifle, Exotic]","[Last Hope, Sidearm, Legendary]"
1,2305843009291975432,2305843009264842296,2305843009260821448230584300926304426023058430...,"[Lincoln Green, Pulse Rifle, Legendary]","[Eternity's Edge, Sword, Legendary]","[The Old Fashioned, Hand Cannon, Legendary]"
2,2305843009290285264,2305843009263044260,2305843009260821448230584300926304426023058430...,"[MIDA Multi-Tool, Scout Rifle, Exotic]","[Prosecutor, Auto Rifle, Legendary]","[Sins of the Past, Rocket Launcher, Legendary]"
3,2305843009278037797,2305843009260821448,2305843009260821448230584300926304426023058430...,"[Vigilance Wing, Pulse Rifle, Exotic]","[The Last Dance, Sidearm, Legendary]","[Sins of the Past, Rocket Launcher, Legendary]"


In [715]:
weapon_1=[]
weapon_2=[]
weapon_3=[]

In [716]:
for playersID in list(weapondf["playersID"]):
    weapon_1.append(result[str(playersID)][0])
    weapon_2.append(result[str(playersID)][1])
    weapon_3.append(result[str(playersID)][2])


In [717]:
weapondf["weapon_1"]=weapon_1
weapondf["weapon_2"]=weapon_2
weapondf["weapon_3"]=weapon_3

In [718]:
weapondf

,targetplayersID,playersID,teamID,weapon_1,weapon_2,weapon_3
0,2305843009264404279,2305843009268897947,2305843009260821448230584300926304426023058430...,"[Galliard-42, Auto Rifle, Legendary]","[MIDA Multi-Tool, Scout Rifle, Exotic]","[Last Hope, Sidearm, Legendary]"
1,2305843009291975432,2305843009264842296,2305843009260821448230584300926304426023058430...,"[Lincoln Green, Pulse Rifle, Legendary]","[Eternity's Edge, Sword, Legendary]","[The Old Fashioned, Hand Cannon, Legendary]"
2,2305843009290285264,2305843009263044260,2305843009260821448230584300926304426023058430...,"[MIDA Multi-Tool, Scout Rifle, Exotic]","[Prosecutor, Auto Rifle, Legendary]","[Sins of the Past, Rocket Launcher, Legendary]"
3,2305843009278037797,2305843009260821448,2305843009260821448230584300926304426023058430...,"[Vigilance Wing, Pulse Rifle, Exotic]","[The Last Dance, Sidearm, Legendary]","[Sins of the Past, Rocket Launcher, Legendary]"
